In [1]:
import math
import torch
import gpytorch
from gpytorch.kernels import RBFKernel
from matplotlib import pyplot as plt
import numpy as np
import tqdm
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
data = np.genfromtxt("../../heron/models/data/gt-M60-F1024.dat")

In [3]:
#data[:,:8] = data[:,:8] + 1e-6*np.random.randn(*data[:,:8].shape)

In [4]:
#data[:,0] = (data[:,0] + 300) / 400.

In [42]:
training_x = torch.tensor(data[:30,[0,1]]*100).float()#.cuda()
training_y = torch.tensor(data[:30,-2]*1e21).float()#.cuda()

In [43]:
from functools import reduce
import operator
def prod(iterable):
    return reduce(operator.mul, iterable)

In [44]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.PeriodicKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [45]:
from gpytorch.priors import LogNormalPrior, NormalPrior, UniformPrior
# Use a positive constraint instead of usual GreaterThan(1e-4) so that LogNormal has support over full range.
likelihood = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.Positive())#.cuda()
model = ExactGPModel(training_x, training_y, likelihood)#.cuda()

In [46]:
import pyro
from pyro.infer.mcmc import NUTS, MCMC

In [47]:
model.mean_module.register_prior("mean_prior", UniformPrior(-1, 1), "constant")
model.covar_module.base_kernel.register_prior("lengthscale_prior", UniformPrior(0.01, 0.5), "lengthscale")
model.covar_module.base_kernel.register_prior("period_length_prior", UniformPrior(0.05, 2.5), "period_length")
model.covar_module.register_prior("outputscale_prior", UniformPrior(1, 2), "outputscale")
likelihood.register_prior("noise_prior", UniformPrior(0.05, 0.3), "noise")

In [48]:
num_samples = 100
warmup_steps = 200

In [49]:
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

def pyro_model(x, y):
    model.pyro_sample_from_prior()
    output = model(x)
    loss = mll.pyro_factor(output, y)
    return y

nuts_kernel = NUTS(pyro_model, adapt_step_size=True)

In [50]:
model.cpu()
likelihood.cpu()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): Positive()
  )
  (noise_prior): UniformPrior(low: 0.05000000074505806, high: 0.30000001192092896)
)

In [51]:
mcmc_run = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps, mp_context="spawn")
mcmc_run.run(training_x, training_y)

Sample: 100%|██████████| 300/300 [29:07,  5.82s/it, step size=4.74e-05, acc. prob=0.756]


In [76]:
model.pyro_load_from_samples(mcmc_run.get_samples())

In [77]:
model.eval()

ExactGPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): Positive()
    )
    (noise_prior): UniformPrior(low: 0.05000000074505806, high: 0.30000001192092896)
  )
  (mean_module): ConstantMean(
    (mean_prior): UniformPrior(low: -1.0, high: 1.0)
  )
  (covar_module): ScaleKernel(
    (base_kernel): PeriodicKernel(
      (raw_lengthscale_constraint): Positive()
      (raw_period_length_constraint): Positive()
      (lengthscale_prior): UniformPrior(low: 0.009999999776482582, high: 0.5)
      (period_length_prior): UniformPrior(low: 0.05000000074505806, high: 2.5)
      (distance_module): Distance()
    )
    (raw_outputscale_constraint): Positive()
    (outputscale_prior): UniformPrior(low: 1.0, high: 2.0)
  )
)

In [78]:
model.eval()
test_x = torch.linspace(0, 1, 101).unsqueeze(-1)
expanded_test_x = test_x.unsqueeze(0).repeat(num_samples, 1, 1)
output = model(expanded_test_x)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 2. Got 2 and 1 in dimension 3 at /pytorch/aten/src/TH/generic/THTensor.cpp:612

In [ ]:
#%%timeit
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    f_preds = model(test_x)
    y_preds = likelihood(f_preds)

In [ ]:
with torch.no_grad():
    lower, upper = y_preds.confidence_region()
    f, ax = plt.subplots(1,1, dpi=300)
    ax.plot(test_x[:,0].cpu().numpy(), f_preds.mean.cpu().numpy()/1e21)
    #samples = y_preds.sample_n(100)
    #for sample in samples:
    #    ax.plot(test_x[:,0].numpy(), sample.numpy()[:,0], alpha=0.01, color='r')
    #    ax.plot(test_x[:,0].numpy(), sample.numpy()[:,1], alpha=0.01, color='b')
    #ax.plot(training_x[:,0].cpu(), training_y.cpu()/1e21, '+')
    ax.fill_between(test_x[:,0].cpu().numpy(), lower.cpu().numpy()/1e21, upper.cpu().numpy()/1e21, alpha=0.5)